In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchvision torchaudio  # Install PyTorch
!pip install opencv-python-headless  # Install OpenCV

In [3]:
!pip install -U yolov5  # Install YOLOv5

In [4]:
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [5]:
import torch
from yolov5 import YOLOv5

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLOv5("yolov5s.pt", device=device)

100%|██████████| 14.1M/14.1M [00:00<00:00, 64.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [8]:
import cv2
import os
from pathlib import Path

def extract_frames(video_path, frames_dir):
    os.makedirs(frames_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(f'{frames_dir}/frame{count:05d}.jpg', frame)
        count += 1
    cap.release()

In [9]:
model = YOLOv5("yolov5s.pt", device="cpu")  # Use "cuda" for GPU

In [13]:
import numpy as np
video_path = '/content/drive/My Drive/Assignment5/Video_for_assignment.mp4'  # Update with your video path
frames_dir = '/content/frames'
annotated_frames_dir = '/content/annotated_frames'

# Extract frames
extract_frames(video_path, frames_dir)

for frame_file in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_file)
    if frame_path.endswith('.jpg'):
        # Perform detection
        results = model.predict(frame_path)
        # Attempt to directly save rendered images might not be straightforward

        # Assuming we need to visualize or access the numpy array of the image
        # Let's use a workaround by accessing the PIL images and converting them
        for i, det in enumerate(results.pred):
            if len(det):
                # If detections exist, render them on the image
                img = results.render()[i]  # Get the PIL image with drawings
                img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)  # Convert PIL image to cv2 image (RGB to BGR)
                cv2.imwrite(os.path.join(annotated_frames_dir, frame_file), img)


In [14]:
import cv2
import os

# Directory containing your annotated frames
annotated_frames_dir = '/content/annotated_frames'

# Output video path
output_video_path = '/content/drive/My Drive/Assignment5/annotated_video.mp4'  # Update this path as necessary

# Assuming all frames are of the same size, get dimensions from the first frame
first_frame_path = os.path.join(annotated_frames_dir, sorted(os.listdir(annotated_frames_dir))[0])
first_frame = cv2.imread(first_frame_path)
height, width, layers = first_frame.shape
size = (width, height)

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, size)  # Adjust FPS as necessary

# Iterate over each frame in the directory
for frame_file in sorted(os.listdir(annotated_frames_dir)):
    frame_path = os.path.join(annotated_frames_dir, frame_file)
    if frame_path.endswith('.jpg'):
        frame = cv2.imread(frame_path)
        out.write(frame)  # Write the frame to the video

# Release everything when job is finished
out.release()